In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.getcwd()
# thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.metric.metric import MetricCalculator
from mesostat.utils.hdf5_io import DataStorage

from lib.gallerosalas.data_fc_db_raw import DataFCDatabase
from lib.analysis.brain_avg_activity import plot_pca1_session, plot_pca1_mouse, calc_metric_mouse_delay

%load_ext autoreload
%autoreload 2

In [ ]:
params = {}
# params['root_path_data']  = gui_fpath("Path to data collection",  './')
params['root_path_data'] = '/media/alyosha/Data/TE_data/yasirdata_raw/'
# params['root_path_data'] = '/home/alyosha/data/yasirdata_raw/'
# params['root_path_data'] = '/media/aleksejs/DataHDD/work/data/yasir/yasirdata_raw'

In [ ]:
dataDB = DataFCDatabase(params)

In [ ]:
print('mice', dataDB.mice)
print('nSessions', len(dataDB.sessions))
print('datatypes', dataDB.get_data_types())
print('nChannel', dataDB.get_nchannels('mou_5'))

In [ ]:
ds = DataStorage('gallerosalas_result_activity.h5')

In [ ]:
mc = MetricCalculator(serial=True, verbose=False)

In [ ]:
dataDB.calc_shortest_distances()

## Compute mean over time for all mice and condition combinations

In [ ]:
dataDB.get_sessions('mou_5')

In [ ]:
for session in dataDB.get_sessions('mou_5'):
    print(session)
    plot_pca1_session(dataDB, 'mou_5', session, trialTypesSelected=('Hit', 'CR'))

In [ ]:
plot_pca1_mouse(dataDB, trialTypesSelected=('Hit', 'CR', 'Miss', 'FA'))

## Effective Rank

In [ ]:
exclQueryLst = [
    {'datatype' : 'bn_trial', 'intervName' : 'PRE'},  # Baseline normalized
    {'mousename' : 'mou_6', 'intervName' : 'REW'},    # No reward for this mouse
]

In [ ]:
trialTypes = ['Hit', 'CR', 'Miss', 'FA']

In [ ]:
calc_metric_mouse_delay(dataDB, mc, ds, 'rank_effective', 's', 'erank_time', skipExisting=False,
                        minTrials=40, dataTypes='auto', trialTypeNames='auto',
                        metricSettings={'allowBadData': True}, exclQueryLst=exclQueryLst, haveDelay=True)

In [ ]:
from lib.analysis.bulk_metrics import plot_metric_bulk

In [ ]:
plot_metric_bulk(ds, 'rank_effective', 'erank_time', xlim=None, ylim=None, yscale=None, verbose=True, xFunc=None)

In [ ]:
ds.list_dsets_pd()

In [ ]:
def _numpy_drop_nan_rows(data2D):
    assert data2D.ndim == 2
    return data2D[~np.isnan(data2D).any(axis=1)]

def _numpy_drop_nan_cols(data2D):
    assert data2D.ndim == 2
    return data2D[:, ~np.isnan(data2D).any(axis=0)]

In [ ]:
aaa = np.array([[1,2,3,4], [5,6,7,8], [9, 10, 11, np.nan]])
_numpy_drop_nan_rows(aaa)

In [ ]:
_numpy_drop_nan_cols(aaa)